<a href="https://colab.research.google.com/github/kt13581358/ai_3/blob/main/exe3(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-huggingface langchain_community faiss-cpu docx2txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
text = "افراد و آگاهی، همه دانش صحیح انتقال با هوشواره معتقدیم در ما"
embed = hf_embedding.embed_query(text)
print(f"طول embedding: {len(embed)}")

طول embedding: 768


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('/content/qhazvin.docx')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)
for chunk in chunks:
    print(chunk)

page_content='یکی از شهرهای بسیار قدیمی در کشورمان از دیرباز تا به الان که در هر دوره ای از تاریخ، نقش مهم و حیاتی داشته است، شهر قزوین است. شهر سر سبز و بسیار زیبای قزوین، به قدری جذاب و بزرگ است که مقصد افراد زیادی چه ایرانی و چه گردشگران خارجی در ایام مختلف سال است. همچنین به دلیل فاصله ی نسبتا کم این شهر با تهران، سالانه افراد زیادی از پایتخت رهسپار این شهر می شوند. ما امروز در الی گشت می خواهیم تا به صورت  

همه چیز درباره ی شهر قزوین' metadata={'source': '/content/qhazvin.docx'}
page_content='همه چیز درباره ی شهر قزوین

شهر قزوین از لحاظ تقسیم بندی های جغرافیایی، مرکز استان هم نام خود یعنی قزوین است. این شهر بسیار خوش آب و هوا، پر است از جاذبه ها و یادگاری های که از تاریخ غنی آن برای ما آیندگان به جای مانده است.' metadata={'source': '/content/qhazvin.docx'}
page_content='البته قزوین بیشترین دلیل شکوفایی اش را تقریبا مدیون این است که هم نزدیک به تهران است و هم نزدیک به استان های شمالی و غربی. همین نزدیکی اش هم باعث تاثیر بر اقلیم آب و هوایی این شهر شده و آن را تبدیل به یکی از بهتر

In [ ]:
from langchain_community.vectorstores import FAISS
import faiss

In [ ]:
index = faiss.IndexFlatL2(len(hf_embedding.embed_query("سالم")))
vector_store = FAISS.from_documents(
    documents = chunks,
    embedding = hf_embedding,
)

query = "غذای سنتی قزوین چیست؟ "
results = vector_store.similarity_search(query, k = 2)

print("\n نتایج معنایی:")
for i, result in enumerate(results, 1):
    print(f"نتیجه {i}:")
    print(result.page_content)
    print(f"منبع: {result.metadata['source']}\n")


 نتایج معنایی:
نتیجه 1:
سفال و سرامیک

محصولات سفال و سرامیک نقش بسیار مهمی در صنایع‌دستی قزوین دارند. آثار سفالینه و سرامیکی با طرح‌ها و نقوش متنوع از جمله سوغاتی‌های محبوب این شهر هستند.

شیرینی‌های محلی

شیرینی‌های محلی مانند شیرینی قندی، پادرازی، نان چای و نان‌بادامی قزوین از جمله سوغاتی‌های خوشمزه و محبوب قزوین هستند.

عطر و ادکلن
منبع: /content/qhazvin.docx

نتیجه 2:
شیرین‌پلو

شیرین پلو یکی از غذاهای سنتی و محبوب استان قزوین است که با طعم شیرین و دلچسب خود، طرفداران زیادی دارد. این غذا از ترکیب برنج، خلال پرتقال، شربت، خلال بادام، خلال پسته، زعفران، کره، سیب زمینی و سینه مرغ تهیه می‌شود.

 آش دوغ قزوین
منبع: /content/qhazvin.docx



فقط یکی از نتایج به دست آمده مستقیم به غذا اشاره دارد.

In [ ]:
index = faiss.IndexFlatL2(len(hf_embedding.embed_query("سالم")))
vector_store = FAISS.from_documents(
    documents = chunks,
    embedding = hf_embedding,
)

query = "غذای سنتی قزوین چیست؟ "
results = vector_store.similarity_search(query, k = 4)

print("\n نتایج معنایی:")
for i, result in enumerate(results, 1):
    print(f"نتیجه {i}:")
    print(result.page_content)
    print(f"منبع: {result.metadata['source']}\n")


 نتایج معنایی:
نتیجه 1:
سفال و سرامیک

محصولات سفال و سرامیک نقش بسیار مهمی در صنایع‌دستی قزوین دارند. آثار سفالینه و سرامیکی با طرح‌ها و نقوش متنوع از جمله سوغاتی‌های محبوب این شهر هستند.

شیرینی‌های محلی

شیرینی‌های محلی مانند شیرینی قندی، پادرازی، نان چای و نان‌بادامی قزوین از جمله سوغاتی‌های خوشمزه و محبوب قزوین هستند.

عطر و ادکلن
منبع: /content/qhazvin.docx

نتیجه 2:
شیرین‌پلو

شیرین پلو یکی از غذاهای سنتی و محبوب استان قزوین است که با طعم شیرین و دلچسب خود، طرفداران زیادی دارد. این غذا از ترکیب برنج، خلال پرتقال، شربت، خلال بادام، خلال پسته، زعفران، کره، سیب زمینی و سینه مرغ تهیه می‌شود.

 آش دوغ قزوین
منبع: /content/qhazvin.docx

نتیجه 3:
معروف‌ترین غذاهای شهر قزوین

شهر قزوین یکی از شهرهای تاریخی و فرهنگی ایران است که دارای غذاهای محلی و خوشمزه‌ای است. غذاهای محلی قزوین شامل قیمه نثار، کوکو شیرین، شیرین‌پلو، حلیم، آش دوغ و دُیماج است.

قیمه نثار

قیمه نثار، معروف‌ترین غذای قزوین که از گوشت، بادام، پسته و زعفران تهیه می‌شود و یکی از غذاهای مجلسی این شهر است. برای پخت این غذا، 

دو نتیجه از نتایج به دست آمده مستقیما به غذاهای محلی اشاره دارد با توجه کد قبلی تقربا 50% نتایج دقیق هستند من فکر می ج نتایج به دست آمده از جستجو با openaiدقت بالاتری دارند.